In [1]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
df = pd.read_csv('data_cleaned.csv')

In [2]:
df.agg(['count', 'size', 'nunique'])

,주호소 및 현병력,진단코드,진단명,진단분류,진단소분류,진단중분류,진료의세부분야,진료과
count,177735,177735,177735,177735,177721,177721,177317,177735
size,177735,177735,177735,177735,177735,177735,177735,177735
nunique,174405,6048,6005,3946,1092,220,163,24


In [3]:
df.head(5)

,주호소 및 현병력,진단코드,진단명,진단분류,진단소분류,진단중분류,진료의세부분야,진료과
0,GB polyp,K828-001,GB polyp,담낭의 기타 명시된 질환,담낭의 기타 질환,"담낭,담도및췌장의장애",췌장.담도질환,소화기내과
1,개인의원에서 시행한 검진 복부 초음파 검사에서 담낭 내 용종이 관찰되어 내원함,K828-001,GB polyp,담낭의 기타 명시된 질환,담낭의 기타 질환,"담낭,담도및췌장의장애",췌장.담도질환,소화기내과
2,AF,I639-001,Cerebral Infarction,상세불명의 뇌경색증,뇌경색(증),대뇌혈관질환,부정맥.실신.돌연사.심박동기.제세동기,심장혈관내과
3,10/31 AF 으로 의뢰. 현재 요양병원 거주,I639-001,Cerebral Infarction,상세불명의 뇌경색증,뇌경색(증),대뇌혈관질환,부정맥.실신.돌연사.심박동기.제세동기,심장혈관내과
4,consult f/u,H2692,"Cataract, unspecified, Bilateral","상세불명의 백내장, 양쪽",기타 백내장,수정체의장애,굴절교정.각막이식.백내장.전안부.각막질환,안과


In [4]:
import re
from konlpy.tag import Komoran
from word2word import Word2word
komoran = Komoran()
ko2en = Word2word('ko', 'en')

def translate_noun(x):
    noun = komoran.nouns(x)
    if noun:
        try:
            translated_noun = ko2en(noun[0], n_best=1)
            return translated_noun[0]
        except KeyError:  # word2word에 없는 단어일 경우
            return None
    return None

def extract_ko_eng(x):
    tokens = x.split()
    ko, eng = [], []
    for token in tokens:
        if re.search(r'[a-zA-Z]', token): # 알파벳이 등장하는 토큰
            eng_token = re.sub(r'[^a-zA-Z/+-]', ' ', token) # 영문 및 / 외 제거, word2vec에 통과시킬 토큰들
            eng_token = eng_token.lower()
            eng += eng_token.split() 
        elif re.search(r'[가-힣]', token): # 한글 토큰
            ko.append(token)
            translated = translate_noun(token) # 명사일 경우 번역 시도
            if translated: 
                eng.append(translated.lower())  
        else: # 기타 기호 토큰
            ko.append(token)
    return ' '.join(ko), ' '.join(eng)

In [5]:
tmp_df = pd.DataFrame(df['주호소 및 현병력'].progress_apply(extract_ko_eng).to_list(), columns=['주호소 및 현병력 한글', '주호소 및 현병력 영문'])
df.insert(1, '주호소 및 현병력 한글', tmp_df['주호소 및 현병력 한글'])
df.insert(2, '주호소 및 현병력 영문', tmp_df['주호소 및 현병력 영문'])
df.head(10)

100%|██████████| 177735/177735 [01:29<00:00, 1994.77it/s]


,주호소 및 현병력,주호소 및 현병력 한글,주호소 및 현병력 영문,진단코드,진단명,진단분류,진단소분류,진단중분류,진료의세부분야,진료과
0,GB polyp,,gb polyp,K828-001,GB polyp,담낭의 기타 명시된 질환,담낭의 기타 질환,"담낭,담도및췌장의장애",췌장.담도질환,소화기내과
1,개인의원에서 시행한 검진 복부 초음파 검사에서 담낭 내 용종이 관찰되어 내원함,개인의원에서 시행한 검진 복부 초음파 검사에서 담낭 내 용종이 관찰되어 내원함,personal procedure checkup abdomen ultrasound ...,K828-001,GB polyp,담낭의 기타 명시된 질환,담낭의 기타 질환,"담낭,담도및췌장의장애",췌장.담도질환,소화기내과
2,AF,,af,I639-001,Cerebral Infarction,상세불명의 뇌경색증,뇌경색(증),대뇌혈관질환,부정맥.실신.돌연사.심박동기.제세동기,심장혈관내과
3,10/31 AF 으로 의뢰. 현재 요양병원 거주,10/31 으로 의뢰. 현재 요양병원 거주,af consult fever habitable,I639-001,Cerebral Infarction,상세불명의 뇌경색증,뇌경색(증),대뇌혈관질환,부정맥.실신.돌연사.심박동기.제세동기,심장혈관내과
4,consult f/u,,consult f/u,H2692,"Cataract, unspecified, Bilateral","상세불명의 백내장, 양쪽",기타 백내장,수정체의장애,굴절교정.각막이식.백내장.전안부.각막질환,안과
5,DM HT. 2019년 4월부터 좌안 시력저하. 양안 산동 good,2019년 4월부터 좌안 시력저하. 양안 산동,dm ht years left sight good,H2692,"Cataract, unspecified, Bilateral","상세불명의 백내장, 양쪽",기타 백내장,수정체의장애,굴절교정.각막이식.백내장.전안부.각막질환,안과
6,평가위해,평가위해,,C241,Malignant neoplasm of ampulla of Vater,파터팽대의 악성 신생물,기타 및 상세불명 담도 부위의 악성 신생물,소화기관의 악성신생물,위암.유방암 및 췌장암.항암치료. 담도암.간암.난소암.자궁암,종양혈액내과
7,"2008년 12.31일 수술, ampulla of vater cancer. 5 FU...","2008년 12.31일 수술, 5 + 6 2009 10 종결. 2013년이 5년째임...",years working surgery ampulla of vater cancer ...,C241,Malignant neoplasm of ampulla of Vater,파터팽대의 악성 신생물,기타 및 상세불명 담도 부위의 악성 신생물,소화기관의 악성신생물,위암.유방암 및 췌장암.항암치료. 담도암.간암.난소암.자궁암,종양혈액내과
8,sh. pain.,,sh pain,S4230,"Fracture of shaft of humerus, closed","상완골 몸통의 골절, 폐쇄성",어깨 및 팔죽지의 골절,어깨및위팔의손상,어깨.팔꿈치 관절:오십견.회전근개 질환.테니스 엘보우.스포츠 손상 및 외상.관절경 ...,정형외과
9,"CRIF using polaros IM nail, Lt 2019.5.22. arm ...","2019.5.22. 2 18.06.17, 18.07.12 , 과거력으로 본원 중임....",crif using polaros im nail lt arm sling for we...,S4230,"Fracture of shaft of humerus, closed","상완골 몸통의 골절, 폐쇄성",어깨 및 팔죽지의 골절,어깨및위팔의손상,어깨.팔꿈치 관절:오십견.회전근개 질환.테니스 엘보우.스포츠 손상 및 외상.관절경 ...,정형외과


In [6]:
df.agg(['count', 'size', 'nunique'])

,주호소 및 현병력,주호소 및 현병력 한글,주호소 및 현병력 영문,진단코드,진단명,진단분류,진단소분류,진단중분류,진료의세부분야,진료과
count,177735,177735,177735,177735,177735,177735,177721,177721,177317,177735
size,177735,177735,177735,177735,177735,177735,177735,177735,177735,177735
nunique,174405,136104,146242,6048,6005,3946,1092,220,163,24


In [7]:
df.to_csv('data_koeng_splited_translated.csv', index=False)

In [7]:
df = pd.read_csv('data_koeng_splited_translated.csv')
length_series = df['주호소 및 현병력'].apply(lambda x: len(x))
length_series.quantile([0.5, 0.75, 0.9])

0.50     41.0
0.75     94.0
0.90    176.0
Name: 주호소 및 현병력, dtype: float64

In [9]:
df = df.fillna('')
length_series = df['주호소 및 현병력 영문'].apply(lambda x: len(x))
length_series.quantile([0.5, 0.75, 0.9])

0.50     45.0
0.75    102.0
0.90    187.0
Name: 주호소 및 현병력 영문, dtype: float64